# Predictive ML Model for Lower Body Injuries

In [49]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.ensemble import BalancedRandomForestClassifier, EasyEnsembleClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced


In [51]:
# This uses the full outer-joined dataset
play_injuries = pd.read_csv('play_injuries_super.csv')

# define the random state for anything using random_state
seed = 42

In [45]:
play_injuries.head()

,PlayerDay,PlayerGame,FieldType,Temperature,PlayerGamePlay,OutdoorStadium,WeatherImpact,InjuryDuration,IsInjured,InjuryType,...,Position_OL,Position_OLB,Position_P,Position_QB,Position_RB,Position_S,Position_SS,Position_T,Position_TE,Position_WR
0,1.0,1.0,1.0,63.0,1.0,1.0,0.0,0.0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,63.0,2.0,1.0,0.0,0.0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,1.0,63.0,3.0,1.0,0.0,0.0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,1.0,63.0,4.0,1.0,0.0,0.0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,1.0,63.0,5.0,1.0,0.0,0.0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


## Prepare test training and testing sets

- The first outcome, y, will be the IsInjured column with a binary outcome. This can be used with the RandomForests for Neural Network Classifications
- The second outcome, z, for a regression analysis with multiple outcomes to classify the types of injuries, and w for the duration of the injury. For duration, it will be important to know that the injury occurred and what type

In [46]:
y = play_injuries.IsInjured
z = play_injuries.InjuryType
w = play_injuries.InjuryDuration

X = play_injuries.drop(columns=['IsInjured', 'InjuryType', 'InjuryDuration'])

For the Machine Learning Model, the following columns are encoded numerically as such: 
- Stadium Types: Outdoor = 1 , or Not Outdoor = 0
- IsInjured: NoInjury = 0, or Any Injury = 1
- Field type: Natural = 0, or synthetic = 1
- WeatherImpact:  Clear and Indoor= 0, Cloudy = 1,  Windy = 2, Hazy/Fog = 3, Rain = 4, Snow = 5 
- InjuryType: NoInjury = 0, Knee = 36, Ankle = 35, Foot = 6 (based on frequency of occurrence)
- Positions have been converted to binary: 1 = True, player plays that position
- PlayType has been converted to binary as well, where 1 is True and indicates that play was active

In [47]:
X.head()

,PlayerDay,PlayerGame,FieldType,Temperature,PlayerGamePlay,OutdoorStadium,WeatherImpact,PlayType_0,PlayType_Extra Point,PlayType_Field Goal,...,Position_OL,Position_OLB,Position_P,Position_QB,Position_RB,Position_S,Position_SS,Position_T,Position_TE,Position_WR
0,1.0,1.0,1.0,63.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,63.0,2.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,1.0,63.0,3.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,1.0,63.0,4.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,1.0,63.0,5.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


Here we look at the outcomes. There are 236000 plays during the 2 seasons, but only 69 plays resulted in injuries of the Foot, Ankle or Knee.
Because this is so imbalanced, stratify is being set = y. 

In [48]:
y.value_counts()

0    236395
1        69
Name: IsInjured, dtype: int64

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed)

## Random Forest Classifier

In [68]:
# Create the Classifier
barf = BalancedRandomForestClassifier(n_estimators=100, random_state=seed)

# Fit the Model
barf.fit(X_train, y_train)

# Calculate the balanced accuracy score
y_pred = barf.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.5531743883041726

In [69]:
# Display the confusion Matrix
confusion_matrix(y_test, y_pred)

array([[35834, 23264],
       [    9,     9]], dtype=int64)

In [70]:
# Print imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.61      0.50      0.75      0.55      0.31     59098
          1       0.00      0.50      0.61      0.00      0.55      0.30        18

avg / total       1.00      0.61      0.50      0.75      0.55      0.31     59116



In [71]:
# List features by order of importance
sorted(zip(barf.feature_importances_, X.columns), reverse=True)

[(0.16861192588587892, 'PlayerGame'),
 (0.15036128154432005, 'PlayerDay'),
 (0.1416428354058179, 'Temperature'),
 (0.12190357221285214, 'PlayerGamePlay'),
 (0.05151982863592106, 'WeatherImpact'),
 (0.0365032870924247, 'FieldType'),
 (0.027827553428202163, 'Position_OLB'),
 (0.026024119508144736, 'Position_WR'),
 (0.025677549280645487, 'PlayType_Rush'),
 (0.025518806441827552, 'OutdoorStadium'),
 (0.025254531907197477, 'PlayType_Pass'),
 (0.01707707032259712, 'Position_FS'),
 (0.015675504954952974, 'Position_SS'),
 (0.01376582738539432, 'Position_G'),
 (0.01317227772163271, 'PlayType_Punt'),
 (0.01309720987509238, 'Position_CB'),
 (0.01249136421294299, 'Position_DE'),
 (0.011186785812341143, 'Position_DT'),
 (0.011115703599857949, 'Position_T'),
 (0.010805710405672377, 'Position_RB'),
 (0.00969324381039502, 'PlayType_Punt Not Returned'),
 (0.009390120981641844, 'Position_ILB'),
 (0.008523030129860876, 'Position_C'),
 (0.008347830441755685, 'Position_MLB'),
 (0.005727188590219164, 'Posit

- Play types seem to be grouped similar to the frequency, where whether it was a passing play or rushing play, vs. anything else, all of which had lower impact. Going to group the other plays together. 
- Positions are similar - OLB and WR are high impact with the others lower. Before grouping everyone else, going to change this to RosterPosition, rather than Actual position to see if it improves the accuracy
- These data do not contain anything about the location on the field yet, but we want to group the least impactful features before adding more features
- Weather impact seems to have a higher impact than expected - I want ot look at some survey data and see how players rank the difficulty level of playing in each of those conditions before resetting the scale